In [21]:
import tensorflow as tf 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
from keras import backend as K
import seaborn as sns
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

In [22]:
xy = pd.read_csv('대파Model.csv')

def outliers_iqr(data, name) :
  Q1, Q3 = np.percentile(data[name+"가격"], [25, 75])
  IQR = Q3 - Q1
  lower_bound = Q1 - (IQR * 1.5)
  upper_bound = Q3 + (IQR * 1.5)
  outlier_idx = data[name+"가격"][ (data[name+"가격"] < lower_bound) | (data[name+"가격"] > upper_bound) ].index
  data.drop(outlier_idx, axis=0, inplace=True)
  data.reset_index(drop=True, inplace=True)
  return data


xy = outliers_iqr(xy,"대파")



In [23]:
%matplotlib notebook
plt.plot(xy['대파가격']) # 전체 종가
xy.index = xy['날짜']
xy = xy.drop(['날짜'],axis=1)

<IPython.core.display.Javascript object>

In [24]:
seqLength = 21 #window size 
dataDim = 27 # 시가, 고가, 저가, 거래량 , 종가
hiddenDim = 100
outputDim = 1
lr = 0.01
iterations = 500

trainSize = int(len(xy)*0.7)
trainSet = xy[0:trainSize]
testSet = xy[trainSize-seqLength:]

scaler = MinMaxScaler()

trainSet = scaler.fit_transform(trainSet)
testSet = scaler.transform(testSet)


In [25]:

# 7일간의 5가지 데이터(시가, 종가, 고가, 저가, 거래량)를 받아와서 
# 바로 다음 날의 종가를 예측하는 모델로 구성

def buildDataSet(timeSeries, seqLength):
    xdata = []
    ydata = [] 
    a = len(timeSeries)
    for i in range(0, a-seqLength):
        tx = timeSeries[i:i+seqLength,:-1]
        ty = timeSeries[i+seqLength,[-1]]
        xdata.append(tx)
        ydata.append(ty)
    return np.array(xdata), np.array(ydata)
    
trainX, trainY=buildDataSet(trainSet, seqLength)
testX, testY=buildDataSet(testSet, seqLength)

In [26]:
# 모델 구성 

# First, let's define a RNN Cell, as a layer subclass.
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()

model.add(layers.SimpleRNN(units=10, activation='tanh',input_shape=[21,26]))

model.add(layers.Dense(1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 10)                370       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 381
Trainable params: 381
Non-trainable params: 0
_________________________________________________________________


In [27]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

model.compile(loss='mse', optimizer=Adam(learning_rate=0.001), metrics=['mse'])

# 모델 트레이닝 
hist = model.fit(trainX, trainY, epochs=500, batch_size=128)

Epoch 1/500
12/12 [==============================] - 1s 2ms/step - loss: 0.4872 - mse: 0.4872
Epoch 2/500
12/12 [==============================] - 0s 2ms/step - loss: 0.2274 - mse: 0.2274
Epoch 3/500
12/12 [==============================] - 0s 2ms/step - loss: 0.1223 - mse: 0.1223
Epoch 4/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0846 - mse: 0.0846
Epoch 5/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0659 - mse: 0.0659
Epoch 6/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0559 - mse: 0.0559
Epoch 7/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0483 - mse: 0.0483
Epoch 8/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0427 - mse: 0.0427
Epoch 9/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0384 - mse: 0.0384
Epoch 10/500
12/12 [==============================] - 0s 2ms/step - loss: 0.0351 - mse: 0.0351
Epoch 11/500
12/12 [==============================] - 0s 2m

In [31]:
res = model.evaluate(testX, testY, batch_size=16)

model.summary()

41/41 [==============================] - 0s 848us/step - loss: 1.0129 - mse: 1.0129
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 10)                370       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 381
Trainable params: 381
Non-trainable params: 0
_________________________________________________________________


In [29]:
#7 모델 사용
xhat = testX

yhat = model.predict(xhat)

# yhat = scaler.inverse_transform(yhat)
# testY = scaler.inverse_transform(testY)

In [30]:
%matplotlib notebook
plt.figure(figsize=(20,10))
plt.plot(yhat, label = "Green_Onion_RNN")
plt.plot(testY,label = "Real")

plt.legend(prop={'size': 20})
plt.savefig('결과/대파모델.png')


<IPython.core.display.Javascript object>